In [1]:
# GRETEL - WHEN BRAIN IS FUNCTIONING: Figure out what's wrong with the ID and then insert the rest of the data

In [2]:
# Import Dependencies
import pandas as pd
import numpy as np
import re
import spacy
import sqlite3
from spacy.attrs import ORTH, LEMMA, NORM, TAG
from datetime import datetime, timedelta
from spacy.attrs import ORTH, LEMMA, NORM, TAG
from clean import replace_emoticons, clean_text, clean_tweet
from joblib import load

In [3]:
# Load machine learning prediction model
model = load("twitter_model.joblib")

In [4]:
# Connect to database and create twitter table
conn = sqlite3.connect("politics_db.db")
conn.execute("""CREATE TABLE "twitter" (
            "id" INTEGER UNIQUE,
            "tweet" TEXT, 
            "tweet_date" TEXT,
            "retweet" INTEGER,
            "favorite" INTEGER,
            "username" TEXT,
            "sentiment" INTEGER     
            )""")

In [5]:
# GRETEL - GET THESE NAMES FROM THE DATABASE

usernames = ["@JoeBiden", "@BernieSanders", "@ewarren", "@KamalaHarris"]

In [6]:
for username in usernames:

    # Import candidate's tweets into a dataframe
    twitter_df = pd.read_csv(f"data/{username}_clean.csv", error_bad_lines=False)

    # Drop old index
    twitter_df = twitter_df.loc[:, ~twitter_df.columns.str.contains('^Unnamed')]
    
    # Apply model to tweet column to get sentiment prediction
    twitter_df["sentiment"] = model.predict(twitter_df["tweet"].values.astype("U"))
    
    # Create username column with candidate's username
    twitter_df["username"] = username
    
    # Set index to be id column
    twitter_df = twitter_df.set_index('id')
    
    # Add candidate's tweets to twitter table
    twitter_df.to_sql("twitter", conn, if_exists="append") 

IntegrityError: UNIQUE constraint failed: twitter.id